# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [1]:
# Check version
!pip install torch==0.4.1

  Using cached https://files.pythonhosted.org/packages/49/0e/e382bcf1a6ae8225f50b99cc26effa2d4cc6d66975ccf3fa9590efcbedce/torch-0.4.1-cp36-cp36m-manylinux1_x86_64.whl
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Imports
import torch
from torch import nn
from torchvision import models
print(torch.__version__)

0.4.1


In [4]:
# Load and Modify Models

resnet152 = models.resnet152(pretrained=True)
vgg19 = models.vgg19(pretrained=True)
densenet161 = models.densenet161(pretrained=True)

# Add attributes to models
resnet152.name = 'ResNet_152'
resnet152.last_layer_attr = 'fc'

vgg19.name = 'VGG_19'
vgg19.last_layer_attr = 'classifier'

densenet161.name = 'DenseNet_161'
densenet161.last_layer_attr = 'classifier'

# List of Models and its last layer
transfer_models = [resnet152,
                    vgg19,
                    densenet161]

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:15<00:00, 15237892.98it/s]
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:49<00:00, 12036216.15it/s]
/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.torch/models/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:04<00:00, 23977525.58it/s]


In [6]:
# Re-defines last layers
def redefine_last_layer(model, n_inputs, n_outputs):
    """
    Changes models last layer.
    """
    # Check if returns a list
    attr = getattr(model, model.last_layer_attr)
    if isinstance(attr, nn.Sequential):
        attr[-1] = nn.Linear(n_inputs, n_outputs)
    else:
        attr = nn.Linear(n_inputs, n_outputs)
    
    # Modifies last layer attributes
    setattr(model, model.last_layer_attr, attr)

# For all models
for model in transfer_models:
    try:
        n_in = getattr(model, model.last_layer_attr).in_features
    except AttributeError:
        n_in = getattr(model, model.last_layer_attr)[-1].in_features
    n_out = 102
    redefine_last_layer(model, n_in, n_out)
    print(model.name + ": " + str(getattr(model, model.last_layer_attr)))


ResNet_152: Linear(in_features=2048, out_features=102, bias=True)
VGG_19: Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=102, bias=True)
)
DenseNet_161: Linear(in_features=2208, out_features=102, bias=True)


In [7]:
# loads a checkpoint and rebuilds the model
def load_model(filepath):
    """
    Loads models from transfer learning.
    """
    checkpoint = torch.load(filepath, map_location='cpu')
    if 'VGG_19' in filepath:
        model = vgg19
    elif 'DenseNet_161' in filepath:
        model = densenet161
    elif 'ResNet_152' in filepath:
        model = resnet152
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = {idx:class_ for class_, idx in model.class_to_idx.items()}
    model.name = checkpoint['name']
    model.optimizer_state  = checkpoint['optimizer_state']
    model.eval()
    
    return model

In [8]:
# Load your model to this variable
model = load_model('checkpoint_ResNet_152.pth')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]